In [1]:
# ===============================
# Cell 1: Imports & Setup
# ===============================
!pip install opencv-python
!pip install tensorflow


import cv2
import time
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import load_model

# Avoid GPU memory pre-allocation (optional)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        pass

print("✅ Imports done")



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 331.7/331.7 MB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 71.9/71.9 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 2.9/2.9 MB 5.2 MB/s eta 0:00:00
     -------------------------------------- 135.8/135.8 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 6.0 MB/s eta 0:00:00
     -------------------------------------- 206.3/206.3 kB 6.3 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 64.7/64.7 kB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 26.4/26.4 MB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 4.6/4.6 MB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 8.9 MB/s eta 0:00:00
     -------------------------------------- 435.7/435.7 kB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 72.5/72.5


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Imports done


In [2]:
# ===============================
# Cell 2: Load Model & Labels
# ===============================
# Path to your saved model (.h5)
model_path = "C:\\Users\\adity\\OneDrive\\Desktop\\ai moodmate\\models\\fer_tl.h5"

# Load model
model = load_model(model_path)
print("✅ Model loaded:", model_path)

# Labels (FER2013 common order)
labels = ["angry","disgust","fear","happy","sad","surprise","neutral"]

# Get model input details
input_shape = model.input_shape
img_size = input_shape[1]  # usually 48
channels = input_shape[-1] # 1 or 3
print("Model expects:", input_shape)


✅ Model loaded: C:\Users\adity\OneDrive\Desktop\ai moodmate\models\fer_tl.h5
Model expects: (None, 96, 96, 3)


In [ ]:

def preprocess_face(face_img, img_size=48, model_channels=1):
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (img_size, img_size))
    arr = resized.astype("float32") / 255.0
    if model_channels == 1:
        arr = np.expand_dims(arr, -1)       # (48,48,1)
    else:
        arr = np.stack([arr, arr, arr], -1) # (48,48,3)
    return arr

def aggregate_probs(prob_deque, labels):
    if not prob_deque:
        return None, 0.0
    sum_probs = np.sum([p for (_, p) in prob_deque], axis=0)
    sum_probs = sum_probs / (np.sum(sum_probs) + 1e-9)
    idx = np.argmax(sum_probs)
    return labels[idx], float(sum_probs[idx])



In [5]:

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
if face_cascade.empty():
    raise RuntimeError("❌ Haar cascade not loaded!")

cap = cv2.VideoCapture(0)  # change index if multiple webcams
if not cap.isOpened():
    raise RuntimeError("❌ Could not open webcam!")

interval = 2.0  # seconds for smoothing
prob_deque = deque()

print("✅ Webcam started — press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(40,40))
    now = time.time()

    # Remove old predictions
    while prob_deque and (now - prob_deque[0][0] > interval):
        prob_deque.popleft()

    for (x,y,w,h) in faces:
        face_patch = frame[y:y+h, x:x+w]
        inp = preprocess_face(face_patch, img_size=img_size, model_channels=channels)
        inp_batch = np.expand_dims(inp, 0)
        
        probs = model.predict(inp_batch, verbose=0)[0]
        label = labels[np.argmax(probs)]
        conf  = np.max(probs)

        prob_deque.append((now, probs))

        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame, f"{label} {conf*100:.1f}%", (x,y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    agg_label, agg_conf = aggregate_probs(prob_deque, labels)
    if agg_label:
        cv2.putText(frame, f"Smoothed: {agg_label} {agg_conf*100:.1f}%", 
                    (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)

    cv2.imshow("Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


✅ Webcam started — press 'q' to quit
